## Step 1
Inserting all necessary packages needed for this step

In [ ]:
from utils import reshape_loaded_ds
from pathlib import Path
from movement.io import load_poses

Defining the file path for the data

In [ ]:
data_folder = Path(r"C:\Users\Yousuf\Desktop\Project\Data")
print(data_folder)

In [ ]:
female_1_name = "220719_SB019_FM001_female1_2022-07-19-164002DLC_resnet50_shanice_allNov29shuffle1_196000_filtered.csv"
female_1_path = data_folder / female_1_name
print(female_1_path)

Reshaping the data in order to load into movement appropriately

In [ ]:
ds = load_poses.from_dlc_file(female_1_path, fps=50)
all_keypoints = ds.keypoints.values

resident_keypoints = []
intruder_keypoints = []

for kp in all_keypoints:
    if kp.startswith("resident"):
        resident_keypoints.append(kp.split("_")[1])
    else:
        intruder_keypoints.append(kp.split("_")[1])
ds_new = reshape_loaded_ds(ds,["resident","intruder"],intruder_keypoints)
ds_new

Selecting relevant time period from data

In [ ]:
ds_new  = ds_new.sel(time=slice(34, 60.8))
ds_new

Plotting **position** data over time for all individuals and keypoints. 

In [ ]:
position = ds_new.position
position

In [ ]:
position.plot.line(x="time", row="keypoints",  hue="individuals", col="space", aspect=2, size=2.5)

Plotting **confidence** data with xarray for all individuals and keypoints

In [ ]:
confidence = ds_new.confidence
confidence

In [ ]:
confidence.plot.line(x="time", row="keypoints",  hue="individuals", aspect=2, size=2.5)

## Step 2

Plotting confidence histograms for each keypoint

In [ ]:
from matplotlib import pyplot as plt

for individual in confidence.individuals:
    da = confidence.sel(individuals=individual)
    da.plot.hist(bins=50, alpha=0.5, label=individual)
ax = plt.gca()
ax.legend()



In [ ]:
for keypoint in confidence.keypoints:
    da = confidence.sel(keypoints=keypoint)
    da.plot.hist(bins=50, alpha=0.5, label=keypoint)
ax = plt.gca()
ax.legend()

In [ ]:

ds_new.update({"position": ds_new.move.filter_by_confidence()})